# 2.2 数据预处理

In [1]:
#Python中常用的数据分析工具是pandas软件包
#本节主要使用pandas预处理原始数据，并将原始数据转换为张量格式

## 2.2.1 读取数据集

In [20]:
#首先创建一个人工数据集，并存储在CSV文件中
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms, Alley, Price\n')  #列名
    f.write('NA,Pave,140000\n') #每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [21]:
#从创建的csv文件中加载原始数据集，导入pandas包并调用read_csv函数
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms  Alley   Price
0       NaN   Pave  140000
1       2.0    NaN  106000
2       4.0    NaN  178100
3       NaN    NaN  140000


## 2.2.2 处理缺失值

In [22]:
#NaN代表缺失值，为了处理缺失的数据，典型的方法包含插值法和删除法。其中插值法用一个替代值弥补缺失值，删除法直接忽略掉缺失值。

#通过索引iloc，将data分成inputs和outputs。对于inputs中缺少的值，用同一列均值替换NaN项

inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  Alley
0       3.0   Pave
1       2.0    NaN
2       4.0    NaN
3       3.0    NaN


C:\Users\yangjb\AppData\Local\Temp\ipykernel_15140\133139320.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


In [23]:
#对于inputs中的类别值或离散值，将NaN视为一个类别。由于巷子类型NaN列只接受两种类别值Pave和NaN，pandas可以将两列转换为Alley_Pave和Alley_nan
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms   Alley_Pave   Alley_nan
0       3.0            1           0
1       2.0            0           1
2       4.0            0           1
3       3.0            0           1


## 2.2.3 转换为张量格式

In [25]:
#inputs和outputs中的所有条目都是数值类型，它们可以转换为张量格式
import torch
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([140000, 106000, 178100, 140000]))

## 2.2.4 小结

In [ ]:
#pandas可以与张量兼容
#用pandas处理数据时，可以根据情况选择插值法和删除法

## 2.2.5

In [55]:
#删除缺失值最多的列
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file2 = os.path.join('..', 'data', 'house.csv')
with open(data_file2, 'w') as f:
    f.write('NumRooms, Alley, Price\n')  #列名
    f.write('NA,Pave,140000\n') #每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')
data2 = pd.read_csv(data_file2) 
data2.columns = ['NumRooms', 'Alley', 'Price'] #创建列索引,若不创建则后面无法调取，会导致运行bug
data2

,NumRooms,Alley,Price
0,NaN,Pave,140000
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [56]:
count = 0
count_max = 0
labels = ['NumRooms', 'Alley', 'Price']
for label in labels:
    count = data2[label].isna().sum()
    if count > count_max:
        count = count_max
        flag = label
data_new = data2.drop(flag, axis=1)
data_new

,NumRooms,Price
0,NaN,140000
1,2.0,106000
2,4.0,178100
3,NaN,140000


In [57]:
#将预处理后的数据集转换为张量格式
torch.tensor(data_new.values)

tensor([[       nan, 1.4000e+05],
        [2.0000e+00, 1.0600e+05],
        [4.0000e+00, 1.7810e+05],
        [       nan, 1.4000e+05]], dtype=torch.float64)